In [1]:
import pygame
import numpy as np
import neat
import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIN_WIDTH, WIN_HEIGHT = 600, 400
TIMEOUT_DURATION = 30

In [3]:
class SnakePos:
    def __init__(self, pos, direction):
        self.x = pos[0]
        self.y = pos[1]
        self.direction = direction
        
    def __repr__(self):
        return 'SnakePos({}, {})'.format(str(self.x), str(self.y))

In [4]:
class Snake:
    CELL_WIDTH = 10
    CELL_HEIGHT = 10
    COLOR_HEAD = (255, 100, 0)
    INITIAL_LENGTH = 5
    LEFT = 0
    UP = 1
    RIGHT = 2
    DOWN = 3
    
    def __init__(self, x, y):
        self.length = self.INITIAL_LENGTH
        self.color = [np.random.randint(100, 255), 0, 0]
        self.moves = []
        self.cells = []
        for i in range(self.length):
            cell_pos = x + self.CELL_WIDTH * i, y + self.CELL_HEIGHT
            cell = SnakePos(cell_pos, Snake.LEFT)
            self.cells.append(cell)
            
    @property
    def head(self):
        return self.cells[0]
    
    @property
    def score(self):
        return self.length - Snake.INITIAL_LENGTH
    
    def move(self):
        for cell in self.cells:
            for i, move in enumerate(self.moves):
                if (cell.x, cell.y) == (move.x, move.y):
                    cell.direction = move.direction
                    if cell == self.cells[-1]:
                        self.moves.pop(i)
                    break
                    
            if cell.direction == Snake.LEFT:
                cell.x -= self.CELL_WIDTH
            elif cell.direction == Snake.RIGHT:
                cell.x += self.CELL_WIDTH
            elif cell.direction == Snake.DOWN:
                cell.y += self.CELL_HEIGHT
            elif cell.direction == Snake.UP:
                cell.y -= self.CELL_HEIGHT
                
    def add_move(self, direction):
        pos = self.head.x, self.head.y
        if len(self.moves) == 0 or self.head.direction != direction:
            if (direction + 2) % 4 != self.head.direction:
                self.moves.append(SnakePos(pos, direction))
        
    def add_cell(self):
        self.length += 1
        tail = self.cells[-1]
        cell_pos = [tail.x, tail.y]
        if tail.direction == Snake.RIGHT:
            cell_pos[0] -= Snake.CELL_WIDTH
        elif tail.direction == Snake.LEFT:
            cell_pos[0] += Snake.CELL_WIDTH
        elif tail.direction == Snake.UP:
            cell_pos[1] += Snake.CELL_HEIGHT
        elif tail.direction == Snake.DOWN:
            cell_pos[1] -= Snake.CELL_HEIGHT
        
        cell = SnakePos(cell_pos, tail.direction)
        self.cells.append(cell)
        
    def check_collision(self):
        for cell in self.cells[1:]:
            if (self.head.x, self.head.y) == (cell.x, cell.y):
                return True
        return False
    
    def check_boundaries(self):
        return self.head.x < 0 or self.head.x >= WIN_WIDTH or self.head.y < 0 or self.head.y >= WIN_HEIGHT
    
    def check_food(self, food):
        if (self.head.x, self.head.y) == (food.x, food.y):
            return True
        return False
            
    def draw(self, win):
        for i, cell in enumerate(self.cells):
            cell_surface = pygame.Surface((self.CELL_WIDTH, self.CELL_HEIGHT))
            cell_surface.fill(self.color)
            win.blit(cell_surface, (cell.x, cell.y))
            
        for move in self.moves:
            move_surface = pygame.Surface((self.CELL_WIDTH, self.CELL_HEIGHT))
            cell_surface.fill((255, 255, 255))
            win.blit(cell_surface, (move.x, move.y))

In [5]:
class SnakeFood:
    def __init__(self, snake):
        while True:
            self.x = np.random.randint(1, WIN_WIDTH / 10) * 10
            if self.x != snake.head.x: break
        while True:
            self.y = np.random.randint(1, WIN_HEIGHT / 10) * 10
            if self.y != snake.head.y: break
        self.color = snake.color
        
    def draw(self, win):
        food_surface = pygame.Surface((Snake.CELL_WIDTH, Snake.CELL_HEIGHT))
        food_surface.fill(self.color)
        win.blit(food_surface, (self.x, self.y))

In [ ]:
pygame.init()
font = pygame.font.Font(pygame.font.get_default_font(), 36)
gen_num = 0

def draw_window(snakes, foods, gen_num, seconds, max_score):
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    pygame.display.set_caption("Snake")

    for snake in snakes:
        snake.draw(win)
    
    for food in foods:
        food.draw(win)
    
    text = font.render("Highest Score: " + str(max_score), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))
    
    text = font.render("Duration: " + str(round(seconds, 1)), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 50))
    
    text = font.render("Gen: " + str(gen_num), 1, (255, 255, 255))
    win.blit(text, (10, 10))
    
    text = font.render("Snakes: " + str(len(snakes)), 1, (255, 255, 255))
    win.blit(text, (10, 50))
    
    pygame.display.update()
    
def main(genomes, config):
    global gen_num
    gen_num += 1
    run = True
    timestamp = time.time()
    max_score = 0
    
    nets = []
    ge = []
    snakes = []
    foods = []
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        snake = Snake(300, 300)
        snakes.append(snake)
        g.fitness = 0
        ge.append(g)
        foods.append(SnakeFood(snake))
        
    while run:
        pygame.time.delay(40)
        seconds = time.time() - timestamp
        
        if int(seconds) != 0 and int(seconds) % TIMEOUT_DURATION == 0:
            run = False
            break
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
        
        for x, snake in enumerate(snakes):
            snake.move()
            ge[x].fitness = snake.score

#             if snake.head.direction == Snake.RIGHT and snake.head.y == foods[x].y and snake.head.x < foods[x].x:
#                 ge[x].fitness += 1 # Encourage moving toward food
#             elif snake.head.direction == Snake.LEFT and snake.head.y == foods[x].y and snake.head.x > foods[x].x:
#                 ge[x].fitness += 1 # Encourage moving toward food
#             elif snake.head.direction == Snake.DOWN and snake.head.x == foods[x].x and snake.head.y < foods[x].y:
#                 ge[x].fitness += 1 # Encourage moving toward food
#             elif snake.head.direction == Snake.UP and snake.head.x == foods[x].x and snake.head.y > foods[x].y:
#                 ge[x].fitness += 1 # Encourage moving toward food
                
            score = snake.length - Snake.INITIAL_LENGTH
            if score > max_score:
                max_score = score
            
            dx, dy = np.abs(np.array((snake.head.x, snake.head.y)) - (foods[x].x, foods[x].y))
            output = nets[x].activate((snake.head.direction, snake.head.x, snake.head.y, dx, dy))
            output_direction = np.argmax(output)
            snake.add_move(output_direction)
            
#             if int(seconds) > 0 and int(seconds) % 10 == 0:
#                 ge[x].fitness -= 10
            
            if snake.check_collision():
#                 ge[x].fitness -= 50
                snakes.pop(x)
                nets.pop(x)
                ge.pop(x)
            elif snake.check_boundaries():
#                 ge[x].fitness -= 5
                snakes.pop(x)
                nets.pop(x)
                ge.pop(x)
                
            if len(snakes) == 0:
                run = False

            elif snake.check_food(foods[x]):
                snake.add_cell()
                foods[x] = SnakeFood(snake)

        if run:
            draw_window(snakes, foods, gen_num, seconds, max_score)

def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    p.add_reporter(neat.StatisticsReporter())
    
    winner = p.run(main, 50)
    pygame.quit()
    
config_path = 'config-feedforward.txt'
run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 0.00600 stdev: 0.07723
Best fitness: 1.00000 - size: (9, 45) - species 9 - id 9
Average adjusted fitness: 0.006
Mean genetic distance 3.328, standard deviation 0.189
Population of 1096 members in 500 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2      0.0    0.000     0
     2    0     2      0.0    0.000     0
     3    0     2      0.0    0.000     0
     4    0     2      0.0    0.000     0
     5    0     2      0.0    0.000     0
     6    0     2      0.0    0.000     0
     7    0     2      0.0    0.000     0
     8    0     2      0.0    0.000     0
     9    0    34      1.0    1.000     0
    10    0     2      0.0    0.000     0
    11    0     2      0.0    0.000     0
    12    0     2      0.0    0.000     0
    13    0     2      0.0    0.000     0
    14    0     2      0.0    0.000     0
    15    0     2      0.0    0.000     0
    16  